In [1]:
import pandas as pd
from sklearn.impute import KNNImputer

ratings = pd.read_csv('data/ml-32m/ratings.csv', dtype={'userId': 'int64', 'movieId': 'int64', 'rating': 'float64'})
ratings = ratings.head(100000)
movies = pd.read_csv('data/ml-32m/movies.csv', dtype={'movieId': 'int64', 'title': 'str', 'genres': 'str'})

In [3]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,17,4.0,944249077
1,1,25,1.0,944250228
2,1,29,2.0,943230976
3,1,30,5.0,944249077
4,1,32,5.0,943228858


In [4]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
print(ratings.isnull().sum())
print()
print(movies.isnull().sum())

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

movieId    0
title      0
genres     0
dtype: int64


In [16]:
data = pd.merge(ratings, movies, on='movieId')
data.head()

,userId,movieId,rating,timestamp,title,genres
0,1,17,4.0,944249077,Sense and Sensibility (1995),Drama|Romance
1,1,25,1.0,944250228,Leaving Las Vegas (1995),Drama|Romance
2,1,29,2.0,943230976,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi
3,1,30,5.0,944249077,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,Crime|Drama
4,1,32,5.0,943228858,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller


In [7]:
print(ratings['userId'].nunique(), "users")
print(ratings['movieId'].nunique(), "movies")

66 users
3965 movies


In [17]:
from scipy.sparse import csr_matrix

user_movie_matrix_sparse = csr_matrix((ratings['rating'], 
                                        (ratings['userId'], 
                                         ratings['movieId'])))
user_movie_matrix_sparse

<627x291486 sparse matrix of type '<class 'numpy.float64'>'
	with 100000 stored elements in Compressed Sparse Row format>

In [20]:
from surprise import Dataset, Reader, SVD, accuracy, KNNBasic, NormalPredictor
from surprise.model_selection import train_test_split

reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(ratings[['userId','movieId','rating']], reader)

trainSet, testSet = train_test_split(data, test_size=0.2, random_state=27)

model = NormalPredictor()
model.fit(trainSet)

predictions = model.test(testSet)
rmse = accuracy.rmse(predictions)
print(f"RMSE: {rmse}")

RMSE: 1.4663
RMSE: 1.466291416842621


In [5]:
import pickle

with open("model_31-10-24.pkl", "rb") as file:
    model = pickle.load(file)

In [4]:
import joblib
model = joblib.load('model.joblib')

KeyboardInterrupt: 

In [21]:
wanted_genres = ['Action','Drama','Thriller','Comedy']
unwanted_genres = ['Fantasy']

min_genre_match = 3

def genre_match(row, genres):
    movie_genres = set(row.split('|'))
    return len(movie_genres.intersection(genres)) >= (min_genre_match if len(genres) >= min_genre_match else len(genres))

filtered_movies = movies[
    movies['genres'].apply(lambda x: genre_match(x, wanted_genres)) & 
    ~movies['genres'].str.contains('|'.join(unwanted_genres))
]

# randomize to prevent oldest first?

# liked_movies = [205383,293]
# liked_of_recommended_movies = filtered_movies[filtered_movies['movieId'].isin(liked_movies)]
# print(liked_of_recommended_movies)

# use tags.csv with AI to find similar movies with similar tags

recommended_movies = []
for movie_id in filtered_movies['movieId'].values:
    predicted_rating = model.predict(uid=0, iid=movie_id)
    recommended_movies.append((movie_id, predicted_rating.est))

recommended_movies.sort(key=lambda x: x[1], reverse=True)

top_recommendations = filtered_movies[filtered_movies['movieId'].isin([x[0] for x in recommended_movies[:10]])]
top_recommendations.to_numpy().tolist()

[[70, 'From Dusk Till Dawn (1996)', 'Action|Comedy|Horror|Thriller'],
 [293,
  'Léon: The Professional (a.k.a. The Professional) (Léon) (1994)',
  'Action|Crime|Drama|Thriller'],
 [2344, 'Runaway Train (1985)', 'Action|Adventure|Drama|Thriller'],
 [2959, 'Fight Club (1999)', 'Action|Crime|Drama|Thriller'],
 [3275, 'Boondock Saints, The (2000)', 'Action|Crime|Drama|Thriller'],
 [3716, 'Fatal Beauty (1987)', 'Action|Comedy|Crime|Drama'],
 [4242, 'Beautiful Creatures (2000)', 'Comedy|Crime|Drama|Thriller'],
 [5666, 'Rules of Attraction, The (2002)', 'Comedy|Drama|Romance|Thriller'],
 [6185, 'Dark Blue (2003)', 'Action|Crime|Drama|Thriller'],
 [7022,
  'Battle Royale (Batoru rowaiaru) (2000)',
  'Action|Drama|Horror|Thriller']]

In [6]:
import pickle

with open('model_31-10-24.pkl', 'wb') as file:
    pickle.dump(model, file)
    
print(f"Model saved!")

Model saved!


In [3]:
import joblib
joblib.dump(model, 'model_31-10-24.joblib')

print(f"Model saved!")

Model saved!
